In [1]:
import os 

In [2]:
os.environ["GOOGLE_API_KEY"]=""

In [20]:
os.environ['GROQ_API_KEY']=''

In [22]:
!pip install -U groq


  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
Using cached distro-1.9.0-py3-none-any.whl (20 kB)


In [96]:
from groq import Groq
client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

chat_completion = client.chat.completions.create(
    messages=[
        {"role": "user", "content": "Explain the importance of fast language models"}
    ],
    model="llama3-8b-8192",
    temperature=0
)

print(chat_completion.choices[0].message.content)

Fast language models, also known as efficient language models or lightweight language models, have gained significant attention in recent years due to their importance in various applications. Here are some reasons why fast language models are crucial:

1. **Scalability**: Fast language models can process large amounts of text data quickly, making them suitable for applications that require real-time processing, such as chatbots, virtual assistants, and language translation systems.
2. **Resource Efficiency**: Traditional language models are computationally expensive and require significant resources (e.g., memory, CPU, and GPU). Fast language models, on the other hand, are designed to be lightweight and can run on lower-end hardware, making them more accessible to a wider range of users.
3. **Edge Computing**: With the increasing adoption of edge computing, fast language models can be deployed on edge devices, such as smartphones, smart home devices, and IoT devices, enabling real-tim

In [97]:
class Agent:
    def __init__(self, client: Groq, system: str = "") -> None:
        self.client = client
        self.system = system
        self.messages: list = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message=""):
        if message:
            self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        completion = client.chat.completions.create(
            model="llama3-70b-8192", messages=self.messages
        )
        return completion.choices[0].message.content

In [98]:
system_prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

wikipedia:
e.g. wikipedia: Django
Returns a summary from searching Wikipedia

Always look things up on Wikipedia if you have the opportunity to do so.

Example session:

Question: What is the capital of France?
Thought: I should look up France on Wikipedia
Action: wikipedia: France
PAUSE 

You will be called again with this:

Observation: France is a country. The capital is Paris.
Thought: I think I have found the answer
Action: Paris.
You should then call the appropriate action and determine the answer from the result

You then output:

Answer: The capital of France is Paris

Example session

Question: What is the mass of Earth times 2?
Thought: I need to find the mass of Earth on Wikipedia
Action: wikipedia : mass of earth
PAUSE

You will be called again with this: 

Observation: mass of earth is 1,1944×10e25

Thought: I need to multiply this by 2
Action: calculate: 5.972e24 * 2
PAUSE

You will be called again with this: 

Observation: 1,1944×10e25

If you have the answer, output it as the Answer.

Answer: The mass of Earth times 2 is 1,1944×10e25.

Now it's your turn:
""".strip()

In [99]:
import re
import httpx
def wikipedia(q):
    return httpx.get("https://en.wikipedia.org/w/api.php", params={
        "action": "query",
        "list": "search",
        "srsearch": q,
        "format": "json"
    }).json()["query"]["search"][0]["snippet"]
#
def calculate(operation: str) -> float:
    return eval(operation)

In [100]:
def loop(max_iterations=20, query: str = ""):

    agent = Agent(client=client, system=system_prompt)

    tools = ["calculate", "wikipedia"]

    next_prompt = query

    i = 0
  
    while i < max_iterations:
        i += 1
        result = agent(next_prompt)
        print(result)

        if "PAUSE" in result and "Action" in result:
            action = re.findall(r"Action: ([a-z_]+): (.+)", result, re.IGNORECASE)
            print(action)
            chosen_tool = action[0][0]
            arg = action[0][1]

            if chosen_tool in tools:
                result_tool = eval(f"{chosen_tool}('{arg}')")
                next_prompt = f"Observation: {result_tool}"

            else:
                next_prompt = "Observation: Tool not found"

            print(next_prompt)
            continue

        if "Answer" in result:
            break

In [104]:
loop(query="Who is Abdul kalam in? in 1 sentence")

Thought: I should look up Abdul Kalam on Wikipedia to find out who he is.

Action: wikipedia: Abdul Kalam 
PAUSE
[('wikipedia', 'Abdul Kalam ')]
Observation: Avul Pakir Jainulabdeen <span class="searchmatch">Abdul</span> <span class="searchmatch">Kalam</span> BR (/ˈəbdʊl <span class="searchmatch">kəˈlɑːm</span>/ ; 15 October 1931 – 27 July 2015) was an Indian aerospace scientist and statesman who served
Thought: I think I have found the answer, Abdul Kalam was an Indian aerospace scientist and statesman.

Action: He was an Indian aerospace scientist and statesman. 
(No action needed, I have the answer)

Answer: Abdul Kalam was an Indian aerospace scientist and statesman.


In [105]:
loop(query="What does decathlon company do?")

Thought: I'm not familiar with the decathlon company, I should look it up on Wikipedia.
Action: wikipedia: Decathlon 
PAUSE
[('wikipedia', 'Decathlon ')]
Observation: The <span class="searchmatch">decathlon</span> is a combined event in athletics consisting of 10 track and field events. The word &quot;<span class="searchmatch">decathlon</span>&quot; was formed, in analogy to the word &quot;pentathlon&quot;
Thought: This doesn't seem to be what I was looking for, it seems to be describing an athletic event.
Thought: I think I need to look up Decathlon again, but this time as a company.
Action: wikipedia: Decathlon (company)
PAUSE
[('wikipedia', 'Decathlon (company)')]
Observation: <span class="searchmatch">Decathlon</span> (French pronunciation: [dekatlɔ̃]) is a French sporting goods retailer. With over 2,080 stores in 78 countries and regions (2023), it is the
Thought: Ah, I've found it! Decathlon is a French sporting goods retailer.
Action: (no action needed, I have the answer)
Answer